In [ ]:
# This script requires github3.py version 0.9.6
# pip install github3.py

import os
import pandas as pd
from github3 import login


GITHUB_API_TOKEN = os.environ.get('GITHUB_API_TOKEN')
gh = login(token=GITHUB_API_TOKEN)
LOCATION = 'Recife'


def test_followers(gh_users_graph):
    ''''''
    assert(gh_users_graph == [
        ('karlafalcao',
            ['tuliohmendes', 'thiagolmm', 'dayvsonlima', 'jbwestphal'])
    ])


def queries_list(location=LOCATION):
    return (
        'location:{} followers:>1'.format(location),
        'location:{} followers:1'.format(location),
        'location:{} followers:0'.format(location),
#         'karlafalcao',
    )


def search_users(query):
    return [u.user for u in gh.search_users(query, sort='followers')]


def fetch_gh_users():
    return [search_users(q) for q in queries_list()]


def create_users_df(batches):
    return pd.concat(
        [pd.DataFrame({'User': u}) for u in batches],
        ignore_index=True)


def followers_iter(gh_user):
    return ','.join([str(u) for u in gh_user.iter_followers()])


def map_f_batch(map_fn, users_iterator):
    return pd.DataFrame({'User': [map_fn(u) for u in users_iterator]})


def create_followers_df(batches):
    return pd.concat(
        [map_f_batch(followers_iter, u) for u in batches],
        ignore_index=True)


In [ ]:
ghusers_batches = fetch_gh_users()
#
gh_users = create_users_df(ghusers_batches)
gh_users.to_csv('gh_users.csv', index=False)
#
gh_followers = create_followers_df(ghusers_batches)
gh_followers.to_csv('gh_followers.csv', index=False, header=False)